# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [768]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk69420"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



Scale the default-node-pool up.

In [899]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk69420/zones/us-central1-c/clusters/fltk-testbed-cluster].


In [210]:
gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 4 --region $REGION --quiet

ERROR: (gcloud.container.clusters.resize) PERMISSION_DENIED: Insufficient project quota to satisfy request: resource "CPUS_ALL_REGIONS": request requires '32.0' and is short '4.0'. project has a quota of '32.0' with '28.0' available. View and manage quotas at https://console.cloud.google.com/iam-admin/quotas?usage=USED&project=test-bed-fltk69420.


: 1

## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [903]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1101 15:40:57.626184   71796 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" has been upgraded. Happy Helming!
NAME: extractor
LAST DEPLOYED: Tue Nov  1 15:40:59 2022
NAMESPACE: test
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [904]:
EXPERIMENT_FILE="../configs/final_boss.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [982]:
helm uninstall -n test experiment-orchestrator

W1101 21:35:08.062292   74970 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


In [983]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

# helm uninstall -n test flearner

pytorchjob.kubeflow.org "trainjob-1a26ce79-ee03-4957-824f-e27b44bef600" deleted
pytorchjob.kubeflow.org "trainjob-1c4a6217-4219-4727-8835-467fe0570920" deleted
pytorchjob.kubeflow.org "trainjob-40f1af9e-e90d-4f69-b065-e51567d35607" deleted
pytorchjob.kubeflow.org "trainjob-45d4a5f7-fda4-4d2d-abca-531f0e0286f3" deleted
pytorchjob.kubeflow.org "trainjob-46dea4c5-802b-4490-821a-20bae642c98e" deleted
pytorchjob.kubeflow.org "trainjob-85e7d703-ca10-41ee-9f49-3785cb33a52e" deleted
pytorchjob.kubeflow.org "trainjob-98a43dad-9a60-4fe7-b7f8-93b37f5e2b74" deleted
pytorchjob.kubeflow.org "trainjob-acd8d0c3-bd71-41c1-a7bb-4d47095fd550" deleted
pytorchjob.kubeflow.org "trainjob-c7be4822-3276-49d2-a8c7-1638eac686a4" deleted
pytorchjob.kubeflow.org "trainjob-e1b9c9dd-cc37-432e-b0ac-6cdd804ccb61" deleted
pytorchjob.kubeflow.org "trainjob-f8f3d7e7-b3d3-4a81-ab90-c05bfdcc3d91" deleted


In [985]:
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1101 21:37:20.885046   75068 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Tue Nov  1 21:37:22 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [987]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

11-01-2022 20:37:30 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
11-01-2022 20:37:30 root         INFO     Starting in cluster mode.
11-01-2022 20:37:30 root         INFO     Starting with experiment replication: 0 with seed: 42
11-01-2022 20:37:30 root         INFO     Starting as Orchestrator
11-01-2022 20:37:30 root         INFO     Starting Orchestrator, initializing resources....
11-01-2022 20:37:30 root         INFO     Loading in cluster configuration file
11-01-2022 20:37:30 root         INFO     Pointing configuration to in cluster configuration.
11-01-2022 20:37:30 root         INFO     Starting cluster manager
11-01-2022 20:37:30 ClusterManager INFO     Spinning up cluster manager...
11-01-2022 20:37:30 ResourceWatchDog INFO     Starting resource watchdog
11-01-2022 20:37:30 ResourceWatchDog INFO     Fetching node information of cluster...
11-01-2022 20:37:30 root         INFO     Starting arrival generator
11-01-2022 20:37:

11-01-2022 20:39:20 Orchestrator INFO     Deploying on cluster: 511f1da6-f69b-4916-9d1e-d16cbaa44004
11-01-2022 20:39:20 Orchestrator INFO     Still alive...
11-01-2022 20:39:23 ResourceWatchDog INFO     Fetching pod information of cluster...
11-01-2022 20:39:25 root         INFO     Waiting for 511f1da6-f69b-4916-9d1e-d16cbaa44004 to complete, 0 pending, 0 arrivals
11-01-2022 20:39:30 root         INFO     Waiting for 511f1da6-f69b-4916-9d1e-d16cbaa44004 to complete, 0 pending, 0 arrivals
11-01-2022 20:39:33 ResourceWatchDog INFO     Fetching pod information of cluster...
11-01-2022 20:39:34 ResourceWatchDog INFO     Fetching pod information of cluster...
11-01-2022 20:39:35 root         INFO     Waiting for 511f1da6-f69b-4916-9d1e-d16cbaa44004 to complete, 0 pending, 0 arrivals
11-01-2022 20:39:40 root         INFO     Waiting for 511f1da6-f69b-4916-9d1e-d16cbaa44004 to complete, 0 pending, 0 arrivals
11-01-2022 20:39:44 ResourceWatchDog INFO     Fetching pod information of cluster..

In [988]:
kubectl get pods -n test

NAME                                                     READY   STATUS      RESTARTS   AGE
fl-extractor-549ff979bf-jzt99                            1/1     Running     0          19h
fl-server                                                1/1     Running     0          153m
nfs-server-nfs-server-provisioner-0                      1/1     Running     0          19h
trainjob-293b09ec-cca0-4dcd-8076-0422a8ef85b8-master-0   0/1     Completed   0          133m
trainjob-293b09ec-cca0-4dcd-8076-0422a8ef85b8-worker-0   0/1     Completed   0          133m
trainjob-293b09ec-cca0-4dcd-8076-0422a8ef85b8-worker-1   0/1     Completed   0          133m
trainjob-293b09ec-cca0-4dcd-8076-0422a8ef85b8-worker-2   0/1     Completed   0          133m
trainjob-37121da4-d7c0-4fa2-8343-0140c45571d3-master-0   0/1     Completed   0          87m
trainjob-37121da4-d7c0-4fa2-8343-0140c45571d3-worker-0   0/1     Completed   0          87m
trainjob-37121da4-d7c0-4fa2-8343-0140c45571d3-worker-1   0/1     Completed 

In [989]:
# To get logs from learners (example)
kubectl logs -n test trainjob-a078f3d3-ccd4-4edc-af4f-7639e5edecc7-master-0


# To get logs from learners (federated learning)
# kubectl logs -n test trainjob-eb056010-7c33-4c46-9559-b197afc7cb84-worker-0

11-01-2022 22:53:07 root         INFO     Loading file config/configuration.fltk.json
11-01-2022 22:53:07 root         INFO     Starting in client mode
11-01-2022 22:53:07 root         INFO     Starting with host=trainjob-a078f3d3-ccd4-4edc-af4f-7639e5edecc7-master-0 and port=23456
11-01-2022 22:53:07 root         INFO     Initializing backend for training process: gloo
11-01-2022 22:53:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
11-01-2022 22:53:10 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
11-01-2022 22:53:10 root         INFO     Starting Creating client with 0
11-01-2022 22:53:10 Client-0-a078f3d3-ccd4-4edc-af4f-7639e5edecc7 INFO     Initializing learning client
11-01-2022 22:53:12 root         INFO     Getting net: Nets.cifar10_cnn
11-01-2022 22:53:12 Client-0-a078f3d3-ccd4-4edc-af4f-7639e5edecc7 INFO     Preparing learner model with d

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [990]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ./logging

tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [993]:
helm uninstall -n test experiment-orchestrator

W1102 05:15:43.541564   78874 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [992]:
helm uninstall extractor -n test

W1102 05:15:39.512720   78858 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [991]:
# This will remove all information and logs as well.
# kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk69420/zones/us-central1-c/clusters/fltk-testbed-cluster].


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk69420/zones/us-central1-c/clusters/fltk-testbed-cluster].
